In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_table('data/user_ratedmovies-timestamps.dat')

In [4]:
df.head()

,userID,movieID,rating,timestamp
0,75,3,1.0,1162160236000
1,75,32,4.5,1162160624000
2,75,110,4.0,1162161008000
3,75,160,2.0,1162160212000
4,75,163,4.0,1162160970000


In [5]:
movies = pd.read_table('data/movies.dat',encoding='latin-1')

In [6]:
movies[['id','title']].head()

,id,title
0,1,Toy story
1,2,Jumanji
2,3,Grumpy Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II


In [21]:
#df_titles = df.set_index('movieID').join(movies[['id','title']].set_index('id'))

In [7]:
df_titles = df.join(movies[['id','title']].set_index('id'), how='left', on='movieID')

In [8]:
df_filtered = df_titles.loc[df_titles['title'].isin(['The Matrix','Alien','The Godfather'])]

In [9]:
df_filtered.head()

,userID,movieID,rating,timestamp,title
29,75,2571,4.5,1162161050000,The Matrix
98,78,858,4.0,1089332870000,The Godfather
149,78,1214,5.0,1083963994000,Alien
286,78,2571,5.0,1083964044000,The Matrix
612,170,2571,3.5,1162208300000,The Matrix


In [10]:
df_titles.loc[ (df_titles['movieID']==3) & (df_titles['userID']==76), ['rating']].empty

True

In [28]:
matrix = np.zeros((df_filtered['title'].nunique(),df_filtered['userID'].nunique()))
distinct_movies = df_filtered['title'].unique()
distinct_users = df_filtered['userID'].unique()

In [12]:
distinct_movies

array(['The Matrix', 'The Godfather', 'Alien'], dtype=object)

In [29]:
for movie in distinct_movies:
    for user in distinct_users:
        if not(df_filtered.loc[ (df_titles['title']==movie) & (df_filtered['userID']==user), ['rating']].empty):
            matrix[np.where(distinct_movies==movie)[0][0]][np.where(distinct_users==user)[0][0]] = df_filtered.loc[ (df_filtered['title']==movie) & (df_filtered['userID']==user), ['rating']].values[0][0]

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
cosine_similarities_movies = cosine_similarity(matrix)
cosine_similarities_movies

array([[1.        , 0.71107806, 0.72501861, 0.        ],
       [0.71107806, 1.        , 0.6626891 , 0.        ],
       [0.72501861, 0.6626891 , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [23]:
cosine_similarities_users = cosine_similarity(matrix.transpose())
cosine_similarities_users

array([[1.        , 0.61545745, 1.        , ..., 0.57735027, 1.        ,
        0.66896473],
       [0.61545745, 1.        , 0.61545745, ..., 0.99493668, 0.61545745,
        0.77769207],
       [1.        , 0.61545745, 1.        , ..., 0.57735027, 1.        ,
        0.66896473],
       ...,
       [0.57735027, 0.99493668, 0.57735027, ..., 1.        , 0.57735027,
        0.81536804],
       [1.        , 0.61545745, 1.        , ..., 0.57735027, 1.        ,
        0.66896473],
       [0.66896473, 0.77769207, 0.66896473, ..., 0.81536804, 0.66896473,
        1.        ]])